In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sn


# Mount Google Drive (if not already mounted)
try:
    drive.mount('/content/drive')
except:
    print("Google Drive already mounted or mount failed.")

# **IMPORTANT**: Replace this with the EXACT path to the file you want to load
file_path_to_load = "/content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt"

try:
    # Load the file using pandas, assuming space-separated values and no header
    df = pd.read_csv(file_path_to_load, header=None, sep='\s+')
    print(f"Successfully loaded file from: {file_path_to_load}")
    print("DataFrame head:")
    display(df.head())
except FileNotFoundError:
    print(f"Error: File not found at {file_path_to_load}. Please check the path in your Google Drive.")
except Exception as e:
    print(f"An error occurred while loading the file: {e}")

<>:20: SyntaxWarning: invalid escape sequence '\s'
<>:20: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3112492813.py:20: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file_path_to_load, header=None, sep='\s+')


Google Drive already mounted or mount failed.
Successfully loaded file from: /content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt
DataFrame head:


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,1.012817,1.022833,1.022028,1.017877,1.023680,1.016974,1.017746,1.019263,1.016417,1.020745,...,1.020981,1.018065,1.019638,1.020017,1.018766,1.019815,1.019290,1.018445,1.019372,1.021171
1,1.018851,1.022380,1.020781,1.020218,1.021344,1.020522,1.019790,1.019216,1.018307,1.017996,...,1.019291,1.019258,1.020736,1.020950,1.020491,1.018685,1.015660,1.014788,1.016499,1.017849
2,1.023127,1.021882,1.019178,1.015861,1.012893,1.016451,1.020331,1.020266,1.021759,1.018649,...,1.020304,1.021516,1.019417,1.019312,1.019448,1.019434,1.019916,1.021041,1.022935,1.022019
3,1.017682,1.018149,1.019854,1.019880,1.019121,1.020479,1.020595,1.016340,1.010611,1.009013,...,1.021295,1.022934,1.022183,1.021637,1.020598,1.018887,1.019161,1.019916,1.019602,1.020735
4,1.019952,1.019616,1.020933,1.023061,1.022242,1.020867,1.021939,1.022300,1.022302,1.022254,...,1.022687,1.023670,1.019899,1.017381,1.020389,1.023884,1.021753,1.019425,1.018896,1.016787


In [3]:
import numpy as np
import pandas as pd

# 경로 기본 설정
HAR_PATH = "/content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/"

# Inertial Signals 불러오기 함수
def load_signals(group):
    signals = []
    signal_types = [
        "total_acc_x", "total_acc_y", "total_acc_z",
        "body_acc_x", "body_acc_y", "body_acc_z",
        "body_gyro_x", "body_gyro_y", "body_gyro_z"
    ]

    for signal in signal_types:
        filename = HAR_PATH + f"{group}/Inertial Signals/{signal}_{group}.txt"
        data = pd.read_csv(filename, sep='\s+', header=None).values
        signals.append(data)

    # (9, samples, 128) → (samples, 128, 9)
    return np.transpose(np.array(signals), (1, 2, 0))

# 라벨 불러오기 함수
def load_labels(group):
    y = pd.read_csv(HAR_PATH + f"{group}/y_{group}.txt", header=None)[0].values
    return y - 1  # 레이블을 0~5로 변환

# ✅ 실제 데이터 로드
X_train = load_signals("train")
X_test = load_signals("test")
y_train = load_labels("train")
y_test = load_labels("test")

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4191593931.py:18: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(filename, sep='\s+', header=None).values


X_train: (7352, 128, 9)
X_test: (2947, 128, 9)
y_train: (7352,)
y_test: (2947,)


In [4]:
#1.간단한 CNN
import tensorflow as tf
from tensorflow.keras import layers, models

def create_simple_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv1D(32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),

        layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),

        layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

num_classes = 6
input_shape = (X_train.shape[1], X_train.shape[2])

model = create_simple_cnn(input_shape, num_classes)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    verbose=1
)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 128, 32)        │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 64, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 64, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 32, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,350 (161.52 KB)

 Trainable params: 40,902 (159.77 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.7516 - loss: 0.6547 - val_accuracy: 0.2672 - val_loss: 1.5292
Epoch 2/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9497 - loss: 0.1343 - val_accuracy: 0.3508 - val_loss: 1.5249
Epoch 3/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9582 - loss: 0.1005 - val_accuracy: 0.7043 - val_loss: 0.8282
Epoch 4/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9547 - loss: 0.1057 - val_accuracy: 0.9150 - val_loss: 0.3559
Epoch 5/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9576 - loss: 0.0998 - val_accuracy: 0.9347 - val_loss: 0.2717
Epoch 6/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9590 - loss: 0.0931 - val_accuracy: 0.9245 - val_loss: 0.3135
Epoch 7/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9651 - loss: 0.0822 - val_accuracy: 0.9082 - val_loss: 0.3434
Epoch 8/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9577 - loss: 0.0888 - val_accuracy: 0.8790 - val_los

In [5]:
#2.LSTM
import tensorflow as tf
from tensorflow.keras import layers, models

# -----------------------------
# ✅ LSTM 모델 정의
# -----------------------------
def create_lstm_model(input_shape, num_classes):
    model = models.Sequential([
        layers.LSTM(64, return_sequences=True, input_shape=input_shape),
        layers.LSTM(64),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# -----------------------------
# ✅ 모델 생성 & 컴파일
# -----------------------------
num_classes = 6
input_shape = (X_train.shape[1], X_train.shape[2])  # (128, 9)

model_lstm = create_lstm_model(input_shape, num_classes)

model_lstm.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

model_lstm.summary()

# -----------------------------
# ✅ 학습
# -----------------------------
history_lstm = model_lstm.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    verbose=1
)

# -----------------------------
# ✅ 평가
# -----------------------------
loss, acc = model_lstm.evaluate(X_test, y_test, verbose=0)
print(f"✅ LSTM Test Accuracy: {acc:.4f}")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128, 64)        │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,518 (220.77 KB)

 Trainable params: 56,518 (220.77 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.3962 - loss: 1.4376 - val_accuracy: 0.6329 - val_loss: 0.9880
Epoch 2/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6307 - loss: 0.8712 - val_accuracy: 0.7029 - val_loss: 0.6829
Epoch 3/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7554 - loss: 0.5848 - val_accuracy: 0.8219 - val_loss: 0.5980
Epoch 4/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8504 - loss: 0.4239 - val_accuracy: 0.8525 - val_loss: 0.5495
Epoch 5/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9006 - loss: 0.2985 - val_accuracy: 0.8994 - val_loss: 0.3986
Epoch 6/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9250 - loss: 0.2212 - val_accuracy: 0.8939 - val_loss: 0.3976
Epoch 7/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9499 - loss: 0.1425 - val_accuracy: 0.8586 - val_loss: 0.5194
Epoch 8/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9498 - loss: 0.1322 - val_accuracy: 0.8960 - v

In [6]:
import numpy as np
import pandas as pd

HAR_PATH = "/content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/"

def load_signals(group):
    signals = []
    signal_types = [
        "total_acc_x", "total_acc_y", "total_acc_z",
        "body_acc_x", "body_acc_y", "body_acc_z",
        "body_gyro_x", "body_gyro_y", "body_gyro_z"
    ]

    for signal in signal_types:
        filename = HAR_PATH + f"{group}/Inertial Signals/{signal}_{group}.txt"
        data = pd.read_csv(filename, sep='\s+', header=None).values
        signals.append(data)

    return np.transpose(np.array(signals), (1, 2, 0))

def load_labels(group):
    y = pd.read_csv(HAR_PATH + f"{group}/y_{group}.txt", header=None)[0].values
    return y - 1

X_train = load_signals("train")
X_test = load_signals("test")
y_train = load_labels("train")
y_test = load_labels("test")

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


<>:16: SyntaxWarning: invalid escape sequence '\s'
<>:16: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1987040389.py:16: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(filename, sep='\s+', header=None).values


X_train: (7352, 128, 9)
X_test: (2947, 128, 9)
y_train: (7352,)
y_test: (2947,)


In [7]:
!ls "/content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI HAR Dataset/"


 __MACOSX  'UCI HAR Dataset'


In [8]:
import shutil

wrong = "/content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/"
correct = "/content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI_HAR_clean/"

shutil.copytree(wrong, correct, dirs_exist_ok=True)

print("✅ 데이터셋 정상 구조 복사 완료:", correct)


✅ 데이터셋 정상 구조 복사 완료: /content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI_HAR_clean/


In [9]:
HAR_PATH = "/content/drive/MyDrive/human+activity+recognition+using+smartphones/UCI_HAR_clean/"

import numpy as np
import pandas as pd

def load_signals(group):
    signals = []
    signals_list = [
        "total_acc_x","total_acc_y","total_acc_z",
        "body_acc_x","body_acc_y","body_acc_z",
        "body_gyro_x","body_gyro_y","body_gyro_z"
    ]

    for s in signals_list:
        filepath = f"{HAR_PATH}/{group}/Inertial Signals/{s}_{group}.txt"
        data = pd.read_csv(filepath, sep="\s+", header=None).values
        signals.append(data)

    return np.transpose(np.array(signals), (1, 2, 0))  # (9,7352,128)→(7352,128,9)

def load_labels(group):
    filepath = f"{HAR_PATH}/{group}/y_{group}.txt"
    return pd.read_csv(filepath, header=None)[0].values - 1

X_train = load_signals("train")
X_test = load_signals("test")
y_train = load_labels("train")
y_test = load_labels("test")

print("✅ 데이터 로드 완료")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)


<>:16: SyntaxWarning: invalid escape sequence '\s'
<>:16: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2568663001.py:16: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(filepath, sep="\s+", header=None).values


✅ 데이터 로드 완료
X_train: (7352, 128, 9)
X_test : (2947, 128, 9)
y_train: (7352,)
y_test : (2947,)


In [10]:
#3.CNN+LSTM
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_lstm_best(input_shape, num_classes):
    model = models.Sequential([
        # CNN Feature Extractors
        layers.Conv1D(64, 3, activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),

        layers.Conv1D(128, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),   # 128→64→32 timesteps

        # LSTM Sequence Learner
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(64, return_sequences=False),

        # Classifier
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

model = create_cnn_lstm_best((128, 9), 6)
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    verbose=1
)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Improved CNN+LSTM Test Accuracy: {acc:.4f}")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 128, 64)        │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 64, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,806 (831.27 KB)

 Trainable params: 212,422 (829.77 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7020 - loss: 0.9309 - val_accuracy: 0.2685 - val_loss: 2.3130
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9472 - loss: 0.1750 - val_accuracy: 0.4296 - val_loss: 2.2066
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9552 - loss: 0.1212 - val_accuracy: 0.5860 - val_loss: 1.3930
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9541 - loss: 0.1091 - val_accuracy: 0.8362 - val_loss: 0.4143
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9560 - loss: 0.1062 - val_accuracy: 0.9130 - val_loss: 0.2698
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9590 - loss: 0.1055 - val_accuracy: 0.9239 - val_loss: 0.2274
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9518 - loss: 0.1037 - val_accuracy: 0.9232 - val_loss: 0.2678
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9597 - loss: 0.0936 - val_accuracy: 0.9062 - v

In [11]:
#4. ResNet
import tensorflow as tf
from tensorflow.keras import layers, models

# -----------------------------
# ResNet Block (채널 자동 정렬 포함)
# -----------------------------
def resnet_block(x, filters):
    shortcut = x

    # 채널 수가 다르면 1x1 conv 로 shortcut 맞춤
    if x.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, 1, padding='same')(shortcut)

    x = layers.Conv1D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv1D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Add()([shortcut, x])
    x = layers.ReLU()(x)
    return x

# -----------------------------
# ✅ ResNet 모델 정의
# -----------------------------
def create_resnet(input_shape=(128, 9), num_classes=6):
    inputs = layers.Input(shape=input_shape)

    # Stem
    x = layers.Conv1D(64, 7, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # ResNet Blocks
    x = resnet_block(x, 64)
    x = layers.MaxPooling1D(2)(x)

    x = resnet_block(x, 128)
    x = layers.MaxPooling1D(2)(x)

    x = resnet_block(x, 256)

    # Classification Head
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# -----------------------------
# ✅ 모델 생성 & 학습
# -----------------------------
model_resnet = create_resnet((128, 9), 6)

model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(0.0007),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_resnet.summary()

history_resnet = model_resnet.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    verbose=1
)

# -----------------------------
# ✅ 평가
# -----------------------------
loss, acc = model_resnet.evaluate(X_test, y_test, verbose=0)
print(f"✅ ResNet Test Accuracy: {acc:.4f}")



Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128, 9)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 128, 64)   │      4,096 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 64)   │        256 │ conv1d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 128, 64)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 128, 64)   │     12,352 │ re_lu[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 64)   │        256 │ conv1d_6[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 128, 64)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 128, 64)   │     12,352 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 64)   │        256 │ conv1d_7[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128, 64)   │          0 │ re_lu[0][0],      │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 128, 64)   │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 64, 64)    │          0 │ re_lu_2[0][0]     │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 64, 128)   │     24,704 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 128)   │        512 │ conv1d_9[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 64, 128)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 64, 128)   │     49,280 │ re_lu_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 64, 128)   │      8,320 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 128)   │        512 │ conv1d_10[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 64, 128)   │          0 │ conv1d_8[0][0],   │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 64, 128)   │          0 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 477,062 (1.82 MB)

 Trainable params: 475,142 (1.81 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 99ms/step - accuracy: 0.8028 - loss: 0.5264 - val_accuracy: 0.2549 - val_loss: 2.4981
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9525 - loss: 0.1150 - val_accuracy: 0.4011 - val_loss: 1.9969
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9509 - loss: 0.1151 - val_accuracy: 0.9252 - val_loss: 0.2966
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9492 - loss: 0.1197 - val_accuracy: 0.9171 - val_loss: 0.2772
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9540 - loss: 0.1012 - val_accuracy: 0.9096 - val_loss: 0.3318
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9544 - loss: 0.0970 - val_accuracy: 0.9096 - val_loss: 0.7346
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9482 - loss: 0.1183 - val_accuracy: 0.9361 - val_loss: 0.5871
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9543 - loss: 0.1061 - val_accuracy: 0.9137 - val_lo

In [14]:
#5. Transformer Encorder
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Fixed Positional Encoding
def positional_encoding(length, depth):
    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(depth)[np.newaxis, :] / depth
    angle_rates = 1 / (10000 ** depths)
    angle_rads = positions * angle_rates
    pe = np.sin(angle_rads) + np.cos(angle_rads)
    return tf.cast(pe, dtype=tf.float32)

# Transformer Encoder block
def transformer_encoder(inputs, num_heads=4, ff_dim=128):
    x = layers.LayerNormalization()(inputs)
    attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(x, x)
    x = layers.Add()([inputs, attn])
    y = layers.LayerNormalization()(x)
    y = layers.Dense(ff_dim, activation='gelu')(y)
    y = layers.Dense(inputs.shape[-1])(y)
    return layers.Add()([x, y])

def create_transformer_model(input_shape=(128, 9), num_classes=6):
    inputs = layers.Input(shape=input_shape)

    pos = positional_encoding(input_shape[0], input_shape[1])   # (128,9)
    pos = tf.expand_dims(pos, axis=0)                           # (1,128,9)
    x = inputs + pos                                            # ✅ shapes now match

    x = transformer_encoder(x, num_heads=4, ff_dim=128)
    x = transformer_encoder(x, num_heads=4, ff_dim=128)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='gelu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)

model_tf = create_transformer_model((128,9), 6)

model_tf.compile(
    optimizer=tf.keras.optimizers.Adam(0.0007),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_tf.summary()

history_tf = model_tf.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    verbose=1
)

loss, acc = model_tf.evaluate(X_test, y_test, verbose=0)
print(f"✅ Transformer Encoder Test Accuracy: {acc:.4f}")



Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 128, 9)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 128, 9)    │          0 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 9)    │         18 │ add_9[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 9)    │      1,413 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_10 (Add)        │ (None, 128, 9)    │          0 │ add_9[0][0],      │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 9)    │         18 │ add_10[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 128, 128)  │      1,280 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 128, 9)    │      1,161 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_11 (Add)        │ (None, 128, 9)    │          0 │ add_10[0][0],     │
│                     │                   │            │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 9)    │         18 │ add_11[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 9)    │      1,413 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, 128, 9)    │          0 │ add_11[0][0],     │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 9)    │         18 │ add_12[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 128, 128)  │      1,280 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 128, 9)    │      1,161 │ dense_16[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_13 (Add)        │ (None, 128, 9)    │          0 │ add_12[0][0],     │
│                     │                   │            │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 9)         │          0 │ add_13[0][0]      │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 128)       │      1,280 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 128)       │          0 │ dense_18[0][0]  

 Total params: 9,834 (38.41 KB)

 Trainable params: 9,834 (38.41 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 16s 91ms/step - accuracy: 0.3015 - loss: 1.5999 - val_accuracy: 0.6037 - val_loss: 1.0819
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5768 - loss: 0.9736 - val_accuracy: 0.6914 - val_loss: 0.7036
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6627 - loss: 0.6868 - val_accuracy: 0.7179 - val_loss: 0.6516
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7003 - loss: 0.6244 - val_accuracy: 0.7362 - val_loss: 0.6284
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7170 - loss: 0.6105 - val_accuracy: 0.7668 - val_loss: 0.6008
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7802 - loss: 0.5194 - val_accuracy: 0.7655 - val_loss: 0.5616
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8006 - loss: 0.4785 - val_accuracy: 0.8049 - val_loss: 0.5241
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8152 - loss: 0.4446 - val_accuracy: 0.8260 - val_los

In [15]:
#6. ResNet + Transformer Encoder
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# -----------------------------
# Positional Encoding (입력 feature 수와 동일하게 유지)
# -----------------------------
def positional_encoding(length, depth):
    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(depth)[np.newaxis, :] / depth
    angle_rates = 1 / (10000 ** depths)
    angle_rads = positions * angle_rates
    pe = np.sin(angle_rads) + np.cos(angle_rads)
    return tf.cast(pe, dtype=tf.float32)   # (length, depth)

# -----------------------------
# ResNet Block
# -----------------------------
def resnet_block(x, filters):
    shortcut = x
    if x.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, 1, padding='same')(shortcut)

    x = layers.Conv1D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv1D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Add()([shortcut, x])
    x = layers.ReLU()(x)
    return x

# -----------------------------
# Transformer Encoder Block
# -----------------------------
def transformer_encoder(x, num_heads=4, ff_dim=128):
    x_norm = layers.LayerNormalization()(x)
    attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x_norm, x_norm)
    x = layers.Add()([x, attn])

    y = layers.LayerNormalization()(x)
    y = layers.Dense(ff_dim, activation='gelu')(y)
    y = layers.Dense(x.shape[-1])(y)
    return layers.Add()([x, y])

# -----------------------------
# ✅ ResNet + Transformer Model
# -----------------------------
def create_resnet_transformer(input_shape=(128, 9), num_classes=6):
    inputs = layers.Input(shape=input_shape)

    # --- ResNet Feature Extractor ---
    x = layers.Conv1D(64, 7, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = resnet_block(x, 64)
    x = layers.MaxPooling1D(2)(x)

    x = resnet_block(x, 128)
    x = layers.MaxPooling1D(2)(x)

    x = resnet_block(x, 256)

    # --- Positional Encoding + Transformer ---
    pe = positional_encoding(x.shape[1], x.shape[2])
    pe = tf.expand_dims(pe, axis=0)
    x = x + pe

    x = transformer_encoder(x, num_heads=4, ff_dim=128)
    x = transformer_encoder(x, num_heads=4, ff_dim=128)

    # --- Classifier ---
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='gelu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)

# -----------------------------
# ✅ 모델 생성 & 학습
# -----------------------------
model_rt = create_resnet_transformer((128, 9), 6)

model_rt.compile(
    optimizer=tf.keras.optimizers.Adam(0.0007),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_rt.summary()

history_rt = model_rt.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    verbose=1
)

# -----------------------------
# ✅ 평가
# -----------------------------
loss, acc = model_rt.evaluate(X_test, y_test, verbose=0)
print(f"✅ ResNet + Transformer Encoder Test Accuracy: {acc:.4f}")


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 128, 9)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 128, 64)   │      4,096 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 64)   │        256 │ conv1d_14[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_7 (ReLU)      │ (None, 128, 64)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_15 (Conv1D)  │ (None, 128, 64)   │     12,352 │ re_lu_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 64)   │        256 │ conv1d_15[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_8 (ReLU)      │ (None, 128, 64)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_16 (Conv1D)  │ (None, 128, 64)   │     12,352 │ re_lu_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 64)   │        256 │ conv1d_16[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_14 (Add)        │ (None, 128, 64)   │          0 │ re_lu_7[0][0],    │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_9 (ReLU)      │ (None, 128, 64)   │          0 │ add_14[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_6     │ (None, 64, 64)    │          0 │ re_lu_9[0][0]     │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 64, 128)   │     24,704 │ max_pooling1d_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 128)   │        512 │ conv1d_18[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_10 (ReLU)     │ (None, 64, 128)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_19 (Conv1D)  │ (None, 64, 128)   │     49,280 │ re_lu_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_17 (Conv1D)  │ (None, 64, 128)   │      8,320 │ max_pooling1d_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 128)   │        512 │ conv1d_19[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_15 (Add)        │ (None, 64, 128)   │          0 │ conv1d_17[0][0],  │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_11 (ReLU)     │ (None, 64, 128)   │          0 │ add_15[0][0]      │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 2,714,758 (10.36 MB)

 Trainable params: 2,712,838 (10.35 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 34s 166ms/step - accuracy: 0.7593 - loss: 0.7879 - val_accuracy: 0.2814 - val_loss: 3.4447
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.9447 - loss: 0.1368 - val_accuracy: 0.3617 - val_loss: 3.7827
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9434 - loss: 0.1248 - val_accuracy: 0.8953 - val_loss: 0.5438
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.9306 - loss: 0.1650 - val_accuracy: 0.9028 - val_loss: 0.4110
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9484 - loss: 0.1218 - val_accuracy: 0.9205 - val_loss: 0.2504
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9491 - loss: 0.1269 - val_accuracy: 0.9361 - val_loss: 0.3608
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9541 - loss: 0.0991 - val_accuracy: 0.8906 - val_loss: 0.2558
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.9514 - loss: 0.1096 - val_accuracy: 0.8892 -